# Implementación de firmas digitales DSA usando python
Juan Felipe Rojas Cendales


In [2]:
!pip install cryptography

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.serialization import PublicFormat, Encoding, load_pem_public_key
from cryptography.exceptions import InvalidSignature

def generate_keys():
    private_key = dsa.generate_private_key(
        key_size=1024,
    )
    public_key = private_key.public_key()
    return private_key, public_key

def sign(private_key, file_path, output_path):
    with open(file_path, "rb") as file:
        file_bytes = file.read()

    signature = private_key.sign(
        file_bytes,
        hashes.SHA256()
    )

    with open(output_path, "w") as file:
        # Signature bytes to HEX
        signature_string = signature.hex()
        file.write(file_bytes.decode('latin-1') + '\n')
        file.write('Firma: ' + signature_string)

    return signature

def save_signature_and_public_key(signature, public_key, output_path):
    with open(output_path, "w") as file:
        # Signature bytes to HEX
        signature_string = signature.hex()
        file.write('Firma: ' + signature_string + '\n')

        # Public key bytes to HEX
        # PEM codification bynary in ASCII.
        # SubjectPublicKeyInfo ASN.1 standar for public keys.
        public_key_string = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo).hex()
        file.write('Llave pública: ' + public_key_string)

def read_signature_and_public_key(input_path):
    with open(input_path, "r") as file:
        signature_string = file.readline().replace('Firma: ', '').strip()
        signature = bytes.fromhex(signature_string)

        public_key_string = file.readline().replace('Llave pública: ', '').strip()
        public_key_bytes = bytes.fromhex(public_key_string)
        public_key = load_pem_public_key(public_key_bytes)

    return signature, public_key

def verify(signature, public_key, file_path, verbose):
    with open(file_path, "rb") as file:
        file_bytes = file.read()
    if verbose:
      print('Llave pública leída: ' + public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo).hex())
      print('Firma leída: ' + signature.hex())
    try:
        public_key.verify(
            signature,
            file_bytes,
            hashes.SHA256()
        )
        print('Verificación: La firma es válida')
        return
    except InvalidSignature:
        print('Verificación: La firma NO es válida')
        return

def main_dsa():
    private_key, public_key = generate_keys()
    print('Llave pública generada: ' + public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo).hex())
    signature = sign(private_key, "/content/drive/MyDrive/Doceavo Semestre/Ciberseguridad/Laboratorios/Laboratorio 1 - Firmas digitales/inscripcion.txt", "/content/drive/MyDrive/Doceavo Semestre/Ciberseguridad/Laboratorios/Laboratorio 1 - Firmas digitales/inscripcionFirmado.txt")
    print('Firma generada: ' + signature.hex())
    save_signature_and_public_key(signature, public_key, "/content/drive/MyDrive/Doceavo Semestre/Ciberseguridad/Laboratorios/Laboratorio 1 - Firmas digitales/publico.txt")
    signature_read, public_key_read = read_signature_and_public_key("/content/drive/MyDrive/Doceavo Semestre/Ciberseguridad/Laboratorios/Laboratorio 1 - Firmas digitales/publico.txt")
    verify(signature_read, public_key_read, "/content/drive/MyDrive/Doceavo Semestre/Ciberseguridad/Laboratorios/Laboratorio 1 - Firmas digitales/inscripcion.txt", True)
    print('---------- Caso de fallo con archivo modificado en un número ----------')
    verify(signature_read, public_key_read, "/content/drive/MyDrive/Doceavo Semestre/Ciberseguridad/Laboratorios/Laboratorio 1 - Firmas digitales/inscripcionModificado.txt", False)

if __name__ == "__main__":
    main_dsa()


Llave pública generada: 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d494942754443434153774742797147534d34344241457767674566416f4742414f744446564f33642b59566d684e615461506d5663663953494b614e5269740a6e4e644139755452613734396b326935507455486333636d474944377a434252746f41775a4e4a734e756633416e665072686d76337a32546e6a496d457137640a533061366a737362534e4369736e636f486a724c78457a6858356a4b4d786947502b774959677a6f5648326b58334d7534734f4d5779573161376b6d5332796c0a392f31442b44376d4b6d4f4641685541376362566c794b554a62626d4a6c5330334d537a54634549517830436759454169336a79496b743454597353654a77620a41386c734c6f766c706164472b667a34585a6d65466e4758596842424d6c2b657631415053306357386175617861486a7762324a6a464757396b534b725449500a697476676a423538533937714b6a41486c4f763472394f4b45394d68784b596d4e76456e46726a393232724c616c52433857594b6c4b4e6d374c38424f3639720a70652b614e6b4c72702b6f54753677435634306a4c454f7a476a554467595541416f4742414a68595855766b476b483061336b4231355658726976506e4c67560a474179762f49